In [2]:
!pip install https://github.com/intersystems-community/intersystems-irispython/releases/download/3.8.0/intersystems_iris-3.8.0-py3-none-any.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.5/119.5 kB 58.7 kB/s eta 0:00:0031m49.5 kB/s eta 0:00:01


In [1]:
import intersystems_iris.dbapi._DBAPI as dbapi
import time
import os

namespace="USER"
port = 1972
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
connection_string = f"{hostname}:{port}/{namespace}"
username = "demo"
password = "demo"

In [2]:
# Note: Ideally conn and cursor should be used with context manager or with try-execpt-finally 
conn = dbapi.connect(connection_string, username, password)
cursor = conn.cursor()

In [3]:
# Users table
users_table_name = "Users"
users_table_definition = """(
    user_id %Library.AutoIncrement PRIMARY KEY,
    name VARCHAR(255),
    date_of_birth DATE,
    medical_conditions TEXT,
    profile_picture BLOB
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {users_table_name} {users_table_definition}")

0

In [3]:
users_table_name = "Users"
users_table_definition = """(
    user_id %Library.AutoIncrement PRIMARY KEY,
    name VARCHAR(255),
    date_of_birth DATE,
    medical_conditions TEXT,
    profile_picture BLOB
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {users_table_name} {users_table_definition}")

# People table
people_table_name = "People"
people_table_definition = """(
    person_id %Library.AutoIncrement PRIMARY KEY,
    user_id INTEGER,
    name VARCHAR(255),
    relationship TEXT,
    description TEXT,
    picture BLOB,
    FOREIGN KEY (user_id) REFERENCES Users(user_id)
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {people_table_name} {people_table_definition}")

# Memories table
memories_table_name = "Memories"
memories_table_definition = """(
    memory_id %Library.AutoIncrement PRIMARY KEY,
    user_id INTEGER,
    person_id INTEGER,
    title TEXT,
    image BLOB,
    description TEXT,
    timestamp DATETIME DEFAULT CURRENT_TIMESTAMP,
    embedding VECTOR(DOUBLE, 1536),
    FOREIGN KEY (user_id) REFERENCES Users(user_id),
    FOREIGN KEY (person_id) REFERENCES People(person_id)
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {memories_table_name} {memories_table_definition}")

0

In [11]:
import base64
import random
import string
from datetime import datetime, timedelta

# Generate random names
def random_name():
    return ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase, k=8))

# Generate random date of birth
def random_dob():
    start_date = datetime(1950, 1, 1)
    end_date = datetime(2000, 12, 31)
    return start_date + (end_date - start_date) * random.random()

# Generate random base64 image data
def random_base64_image():
    return base64.b64encode(bytes([random.randint(0, 255) for _ in range(100)])).decode('utf-8')

# Insert mock users
for _ in range(100):
    name = random_name()
    dob = random_dob().strftime('%Y-%m-%d')
    medical_conditions = random.choice(["Diabetes", "Hypertension", "None"])
    profile_picture = random_base64_image()
    
    cursor.execute(f"""
        INSERT INTO {users_table_name} (name, date_of_birth, medical_conditions, profile_picture)
        VALUES (?, ?, ?, ?)
    """, (name, dob, medical_conditions, profile_picture))

# Fetch all user_ids for use in the People and Memories tables
cursor.execute(f"SELECT user_id FROM {users_table_name}")
user_ids = [row[0] for row in cursor.fetchall()]

# Insert mock people related to users
for _ in range(300):
    user_id = random.choice(user_ids)
    name = random_name()
    relationship = random.choice(["Friend", "Family", "Colleague"])
    description = f"{name} is a {relationship} of user {user_id}."
    picture = random_base64_image()
    

    cursor.execute(f"""
        INSERT INTO {people_table_name} (user_id, name, relationship, description, picture)
        VALUES (?, ?, ?, ?, ?)
    """, (user_id, name, relationship, description, picture))

# Fetch all person_ids for use in the Memories table
cursor.execute(f"SELECT person_id FROM {people_table_name}")
person_ids = [row[0] for row in cursor.fetchall()]

# Insert mock memories for users and people
for _ in range(500):
    user_id = random.choice(user_ids)
    person_id = random.choice(person_ids)
    title = f"Memory with {random_name()}"
    image = random_base64_image()
    description = f"A description of the memory titled '{title}'."
    timestamp = (datetime.now() - timedelta(days=random.randint(1, 1000))).strftime('%Y-%m-%d %H:%M:%S')
    embedding = ','.join([str(random.uniform(-1, 1)) for _ in range(1536)])

    sql = """
    INSERT INTO Memories (user_id, title, image, description, embedding, person_id)
    VALUES (?, ?, ?, ?, TO_VECTOR(?, DOUBLE), ?)
    """
    
    try:
        cursor.execute(sql, (user_id, title, image, description, embedding, person_id))
        conn.commit()        
    except Exception as e:
        conn.rollback()
        print(f"Error inserting memory: {str(e)}")
conn.commit()  # Commit all the changes to the database


In [9]:
tableName = "Demo.Diagnoses"
tableDefinition = "(row_id INTEGER, icd9_code VARCHAR(255), short_title VARCHAR(255), long_title VARCHAR(''))"

In [10]:
try:
    cursor.execute(f"DROP TABLE {tableName}")  
except:
    pass
cursor.execute(f"CREATE TABLE {tableName} {tableDefinition}")

0

In [11]:
## batch update
sql = "Insert into Demo.Diagnoses (row_id, icd9_code,short_title,long_title) values (?, ?, ?,?)"
params = [('0', '0', "entered from batch update 1","entered from batch update 1"), ('0', '0','entered from batch update 2',"entered from batch update 2"), ('0', '0',"entered from batch update 3", "entered from batch update 3")]
cursor.executemany(sql, params) 

3

In [4]:
def insert_user(name, date_of_birth, medical_conditions, profile_picture):
    try:
        cursor.execute("""
            INSERT INTO Users (user_id, name, date_of_birth, medical_conditions, profile_picture) 
            VALUES (?, ?, ?, ?, ?)
        """, [0, name, date_of_birth, medical_conditions, profile_picture])        
        
    except Exception as e:
        print(e)

# Insert a user and get their ID
# TODO: PROMPT ENGINEERING
user_id = insert_user('Tejas Srikanth', '1990-01-01', 'None', 'profile_afadsfasdff')

None


In [14]:
cursor.execute("DELETE * FROM Users")
cursor.fetchall()

DatabaseError: [SQLCODE: <-1>:<Invalid SQL statement>]
[Location: <Prepare>]
[%msg: < IDENTIFIER expected, * found ^DELETE *>]

In [ ]:
users_table_name = "Faces"
users_table_definition = """(
    name VARCHAR(255), 
    relationship VARCHAR(255),
    face_embedding VECTOR(DOUBLE, 512), 
    other_info VARCHAR(512)
)"""

cursor.execute(f"CREATE TABLE IF NOT EXISTS {users_table_name} {users_table_definition}")

In [ ]:
def insert_mem(name, relationship, face_embedding, other_info):
    try:
        cursor.execute("""
            INSERT INTO Faces (name, relationship, face_embedding, other_info) 
            VALUES (?, ?, ?, ?)
        """, [name, relationship, face_embedding, other_info])        
        
    except Exception as e:
        print(e)

# BEGIN: run insert_mem using fake vector embedding
fake_embedding = [0.0] * 512
insert_mem('Jane Doe', 'Friend', fake_embedding, 'Some other info')
# END: run insert_mem using fake vector embedding